In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler 
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV, LassoCV
import warnings

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, f_regression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

import numpy as np
import pandas as pd
import re

Feature Selection (from Caroline)

In [ ]:
train = pd.read_csv('train.csv')

train['baths'] = train['bathrooms_text'].str.findall(r'(\d+.\d+|\d+)')
train['baths'] = train['baths'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else '')
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float) / 100


train['host_is_superhost'] = train['host_is_superhost'].astype('category')
train['neighbourhood_cleansed'] = train['neighbourhood_cleansed'].astype('category')
train['property_type'] = train['property_type'].astype('category')
train['instant_bookable'] = train['instant_bookable'].astype('category')
train['amenities'] = train['amenities'].astype('category')


y = train.loc[:, ['price']]
X = train.loc[:, ['host_location', #maybe
                  'host_response_time', #maybe
                  'host_acceptance_rate', 
                  'host_is_superhost', 
                  'neighbourhood_cleansed', 
                  'property_type', #'room_type'
                  'accommodates', 
                  'baths', 
                  'beds',  #not bedrooms
                  'amenities', # or if it even has amenities, or pools vs hottub or pets
                  'maximum_maximum_nights', 
                  'maximum_nights',
                  'number_of_reviews', 
                  'number_of_reviews_ltm',  #check this
                  'instant_bookable']]

## Daisy Pipe

In [ ]:
numeric = Pipeline(steps=[('impute', SimpleImputer(strategy='median')), 
                 ('polynomial', PolynomialFeatures(degree = 2, include_bias=False)),
                 ('standardize', StandardScaler()),
                 ('percent', SelectPercentile(f_regression, percentile=40))])

categorical = Pipeline(steps=[('impute2', SimpleImputer(strategy='most_frequent')),
                     ('one_hot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
                     ('percent', SelectPercentile(f_regression, percentile=60))
                     ])

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric, num_features),
        ("categorical", categorical, cat_features)
    ]
)


pipe = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('model', DecisionTreeRegressor(random_state=486))
           ]
)

pipe.fit(X_train, y_train)

## Model Loop

In [ ]:
# Used a pipeline to transform the model a little bit, now try and fit a much of models with these features 
# and see the MSE at the end

model = LinearRegression()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print(f'Train MSE for LinReg is: {mean_squared_error(y_train, y_train_pred)}')
print(f'Test MSE for LinReg is: {mean_squared_error(y_test, y_test_pred)}')
print('')


alphas = np.logspace(-6,6,30)
model = RidgeCV(alphas=alphas, cv=10)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print(f'Train MSE for Ridge is: {mean_squared_error(y_train, y_train_pred)}')
print(f'Test MSE for Ridge is: {mean_squared_error(y_test, y_test_pred)}')
print('')



model = LassoCV(cv=10)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print(f'Train MSE for Lasso is: {mean_squared_error(y_train, y_train_pred)}')
print(f'Test MSE for Lasso is: {mean_squared_error(y_test, y_test_pred)}')
print('')

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': RidgeCV(alphas=np.logspace(-6,6,30), cv=10),
    'Lasso Regression': LassoCV(cv=10),
    'Random Forest': RandomForestRegressor(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'SVR': SVR(),
    'KNN': KNeighborsRegressor()
    # ensemble 
    
}

# Create a pipeline for each model
pipelines = {}
for model_name, model in models.items():
    pipelines[model_name] = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

# Fit and evaluate each pipeline
results = {}
for model_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)  # Fit the pipeline
    y_pred = pipeline.predict(X_test)  # Make predictions
    mae = mean_absolute_error(y_test, y_pred)  # Calculate MAE
    results[model_name] = mae

# Print results
for model_name, mae in results.items():
    print(f"{model_name}: MAE = {mae}")

camilla is gonna do bagging